<a href="https://colab.research.google.com/github/TeamBJEJ/Natural-Language-Processing-with-Disaster-tweet/blob/main/NLP_with_disaster_lstm_scratch_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect Kaggle

In [ ]:
!!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        110           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        216           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        615           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       3633           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      29385           False  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       7633           False  
connectx

In [ ]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 36.0MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 58.1MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 64.2MB/s]


# Import libraries

In [ ]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
numpy.random.seed(7)

In [ ]:
ls

drive/  kaggle.json  sample_data/  sample_submission.csv  test.csv  train.csv


# Preparing data

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
X = train_data.drop(['target'], axis = 1)
y = train_data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
print(len(X_train))
print(len(X_test))

6090
1523


In [ ]:
X_train_text = X_train['text']
X_test_text = X_test['text']
X_test_text.head()

3228    Goulburn man Henry Van Bilsen missing: Emergen...
3706    The things we fear most in organizations--fluc...
6957                              @tsunami_esh ?? hey Esh
2887    @POTUS you until you drown by water entering t...
7464    Crawling in my skin\nThese wounds they will no...
Name: text, dtype: object

# Embedding from scratch

In [ ]:
t = Tokenizer()
t.fit_on_texts(X_train_text)
vocab_size = len(t.word_index) + 1
print(vocab_size)

19424


In [ ]:
X_train_encoded = t.texts_to_sequences(X_train_text)
X_test_encoded = t.texts_to_sequences(X_test_text)

In [ ]:
max_len = max(len(l) for l in X_train_encoded)
max_len

33

In [ ]:
X_train_encoded = sequence.pad_sequences(X_train_encoded, maxlen = max_len, padding = 'post')
X_test_encoded = sequence.pad_sequences(X_test_encoded, maxlen = max_len, padding = 'post')

In [ ]:
model = Sequential()
embedding_vocab_length = 32
model.add(Embedding(vocab_size, embedding_vocab_length, input_length=max_len))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test), epochs=5, batch_size=64)
scores = model.evaluate(X_test_encoded, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 33, 32)            621568    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 674,869
Trainable params: 674,869
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
96/96 [==============================] - 4s 26ms/step - loss: 0.6451 - accuracy: 0.6255 - val_loss: 0.4605 - val_accuracy: 0.7997
Epoch 2/5
96/96 [==============================] - 2s 21ms/step - loss: 0.3073 - accuracy: 0.8808 - val_loss: 0.4792 - val_accuracy: 0.7991
Epoch 3/5
96/96 [==============================] - 2s 21ms/step - loss: 0.1492 - accuracy: 0.